In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [2]:
import gradio as gr

In [3]:
# Load env variables, print key prefixes to help w debugging

load_dotenv(override=True)
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
google_key = os.getenv('GOOGLE_API_KEY')
deepseek_key = os.getenv('DEEPSEEK_API_KEY')

if openai_key:
    print(f"OpenAI API Key exists and begins {openai_key[:8]}")
else:
    print("OpenAI key not set.")

if anthropic_key:
    print(f"Anthropic API Key exists and begins {anthropic_key[:7]}")
else:
    print("Anthropic key not set.")

if google_key:
    print(f"Google API Key exists and begins {google_key[:8]}")
else:
    print("Google API Key not set.")

if deepseek_key:
    print(f"DeepSeek API Key exists and begins {deepseek_key[:5]}")
else:
    print("DeepSeek key not set.")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBo
DeepSeek API Key exists and begins sk-18


In [4]:
# Connect to OpenAI, Anthropic, Google

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [5]:
# A generic system message 
system_message = "You are a helpful assistant"

In [ ]:
# Let's wrap a call to GPT-4o-mini in a simple function
